# Industry to Customer Seg
- 업종별 관찰을 시작으로 고객군을 관찰한다
- 기업 입장에서 어떤 고객군을 타깃하는게 효율적인지 관찰한다
- 수치적 지표가 있으면 더 좋다

In [ ]:
import os
import glob
import string
from itertools import combinations

import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import matplotlib as mpl
plt.rc('font', family='NanumGothic')
mpl.rcParams['axes.unicode_minus'] = False
plt.style.use('ggplot')

## Track 2
trend_w_demo.csv: 온라인 가맹점 업종 정보

<u>Description</u>
> YM: 이용 연월(2019년 4월\~2020년 5월)  
Category: 이용 업종  
성별구분: 0(남성), 1(여성)  
연령대: 연령 구간. 알파벳 순서와 연령 순서는 무관
기혼스코어: 카드 이용 고객이 기혼일 확률  
유아자녀스코어: 카드 이용 고객이 유아자녀가 있을 확률  
초등학생자녀스코어: 카드 이용 고객이 초등학생 자녀가 있을 확률  
중고생자녀스코어: 카드 이용 고객이 중고생 자녀가 있을 확률  
대학생자녀스코어: 카드 이용 고객이 대학생 자녀가 있을 확률  
전업주부스코어: 카드 이용 고객이 전업주부일 확률  

In [ ]:
pathList = glob.glob('../../../raw/*')
# score_cols = ['Marriage', 'Youth', 'Elementary', 'Mid-High', 'University', 'Housewife']
score_cols = ['MRG', 'YTH', 'ETY', 'MHC', 'UVC', 'HWF']
ages = [alpha for alpha in string.ascii_uppercase[:8]]
years = [2019, 2020]
genders = [0, 1]
categories = ['디저트', '면세점', '뷰티', '오픈마켓/소셜', '전문몰', 
                 '종합몰', '취미', '할인점', '항공/여행사', '호텔/숙박']

trend_w_demo = (
    pd.read_csv(pathList[-3], engine='python')
    .rename(
        columns = {
            '성별구분':'Sex',
            '연령대':'Age',
            '기혼스코어':'MRG',
            '유아자녀스코어':'YTH',
            '초등학생자녀스코어':'ETY',
            '중고생자녀스코어':'MHC',
            '대학생자녀스코어':'UVC',
            '전업주부스코어':'HWF'
        }
    )
)

data = trend_w_demo.sort_values(by='Category').reset_index(drop=True)
data.loc[:, 'YM'] = data.loc[:, 'YM'].apply(lambda x: int(str(x)[:4]))
# data.loc[:, score_cols] = data.loc[:, score_cols].apply(lambda x: x.apply(lambda x: x[0]))

data.head(3)

## 업종 간의 유사성을 파악하고 고객 Seg별 유사성을 도출해보자

![gini](https://user-images.githubusercontent.com/48649606/95361570-dea97280-0907-11eb-8dbb-09fb130333a0.png)

In [ ]:
def scoreSimilarity(data=data, score_cols=score_cols):
    result_raw = dict(column_1=[], column_2=[], similarity=[])
    for col1, col2 in combinations(score_cols, 2):
        try:
            result_raw['similarity'].append(gini(data=data, col1=col1, col2=col2, inverse=True))
            result_raw['column_1'].append(col1)
            result_raw['column_2'].append(col2)
        except:
            pass
        
    result = (pd.DataFrame(result_raw)
              .sort_values(by='similarity', ascending=False)
              .reset_index(drop=True))
    return result

def gini(data=data, col1='Youth', col2='Elementary', inverse=False):
    quadratic_term = np.square(
        (data[col1] + data[col2]).value_counts() / data.shape[0]
    )
    if inverse:
        return quadratic_term.sum()
    return 1 - quadratic_term.sum()

In [ ]:
def make_seg(data):
    reduced_score_list = list(set(data.columns) - set(['YM', 'Category', 'Sex', 'Age']))
    temp_series = reduced_score_list[0] + '_' + data[reduced_score_list[0]].copy() + '\n'
    for r in reduced_score_list[1:]:
        if r == reduced_score_list[1:][-1]:
            temp_series += r + '_' + data[r]
        else:
            temp_series += r + '_' + data[r] + '\n'
    data['seg'] = temp_series
    return data


def make_rate_change(table2020, table2019):
    segInfo2020 = table2020.groupby('seg').apply(lambda x: len(x)).to_frame('rate')
    segInfo2019 = table2019.groupby('seg').apply(lambda x: len(x)).to_frame('rate')

    diff = list(
        set(segInfo2020.index).union(set(segInfo2019.index)) 
        - set(segInfo2020.index).intersection(set(segInfo2019.index))
    )

    for d in diff:
        if d not in list(segInfo2020.index):
            segInfo2020 = segInfo2020.append(pd.DataFrame(dict(rate=[0]), index=[d]))
            segInfo2020 = segInfo2020.reset_index().sort_values(by='index').set_index('index')
        if d not in list(segInfo2019.index):
            segInfo2019 = segInfo2019.append(pd.DataFrame(dict(rate=[0]), index=[d]))
            segInfo2019 = segInfo2019.reset_index().sort_values(by='index').set_index('index')
    result = ((segInfo2020 / segInfo2019) - 1).reset_index()
    result.columns = ['seg', 'rate']
    return result


def auto_segmentation_per_year(data):
    threshold = scoreSimilarity(data)['similarity'].describe().loc['75%']

    while True:
        simTable = scoreSimilarity(data)
        if (simTable['similarity'][0] < 0.7):
            break
        else:
            if simTable['similarity'][0] >= threshold:
                data = data.drop(simTable['column_1'][0], axis=1)
                simTable = scoreSimilarity(data)
            else:
                break

    table2020 = make_seg(data[data['YM']==2020].reset_index(drop=True))
    table2019 = make_seg(data[data['YM']==2019].reset_index(drop=True))
    
    return table2020, table2019


def plotSomethingGreat(data=data, category='디저트', age='A', save=False):
    male = data[(data['Sex']==0) & (data['Category']==category)].reset_index(drop=True)
    female = data[(data['Sex']==1) & (data['Category']==category)].reset_index(drop=True)
    
    for data_sex in [male, female]:
        sex = '남성' if data_sex['Sex'].unique()[0] == 0 else '여성'
        
        ageFiltered = data_sex[data_sex['Age']==age]
        table2020, table2019 = auto_segmentation_per_year(ageFiltered)
        rateInfo = make_rate_change(table2020, table2019)

        fig, ax = plt.subplots(figsize=(10,5))
        sns.barplot(x='seg', y='rate', data=rateInfo)
        plt.title(f'{age} 연령대 {sex}')
        plt.suptitle(f'코로나19 사태에 따른 {category} 업종 이용변화율(2019년 대비 2020년)')
        plt.xlabel('고객층')
        plt.ylabel('변화율(%p)')
        plt.show()
        
        if save:
            name = f'[{category} 업종 이용변화율] {age} 연령대 {sex}.png'
            path = os.path.join('./plots', name)
            fig.savefig(path, dpi=300)

In [ ]:
for c in categories:
    for a in ages:
        try:
            plotSomethingGreat(data=data, category=c, age=a, save=True)
        except:
            pass